In [1]:
import cv2 
import os 

from moviepy.editor import VideoFileClip
from IPython.display import HTML

# from lane_processor import color_lane
import glob 

# get mtx and dst variables needed for fixing camera distortion
import cv2 
import glob
import numpy as np 

from calibrator import Calibrator
from pipeline_helper import PipelineHelper

calibrator = Calibrator()

# initialize classes for calibration and lane detection
mtx, dist = calibrator.calibrate_camera()


# initialize pipeline with distortion coefficients
pipeline = PipelineHelper(mtx, dist)

def paint_lane(img):
    # set pipeline and lane parameters using passed in image
    if pipeline.roi_params == {}:
        pipeline.set_roi_params(img)

    # correct image distortion 
    undistorted_img = pipeline.undistort_image(img)

    # optimize image for lane detection
    laneline_optimized_img = pipeline.optimize_lane_lines(undistorted_img)

    # roi = pipeline.region_of_interest(laneline_optimized_img)
    # pipeline.show_image("roi", roi)

    # find 4 pts needed for birds-eye perspective transform
    cornerpts = pipeline.find_cornerpts(laneline_optimized_img)
    # convert coords to ints, required for indexing operations based on cornerpts
    cornerpts = [(int(x), int(y)) for x,y in cornerpts]

    # pipeline.plot_cornerpts(cornerpts)
    # print("initial cornerpts ", cornerpts)

    # # optimize corner pts so that birds-eye-transform lanes parallel 
    stop_optimization = False
    
    # loop count for analysis and optimization 
    loops = 0
    
    # transform to binary before perspective transform
    orig_binary = pipeline.image_to_binary(laneline_optimized_img)

    # birds eye binary perspective transform after each optimization loop
    birds_eye_binary = None

    left_diff = float(np.sum(pipeline.lanes.left_lane.diffs))
    right_diff = float(np.sum(pipeline.lanes.right_lane.diffs))

    
    diff_max = 2
    max_moves = 3

    # if diffs are small, ok to change cornerpts
    if abs(left_diff) > diff_max:
        # if fit differencess are large, probably error so set max moves equal 0 to 
        # keep cornerpts as is
        max_moves = 0
        ltc = max_moves
        lbc = max_moves
    else: 
        # allow each left lane cornerpt to move max_moves pixels
        ltc = max_moves
        lbc = max_moves            

    if abs(right_diff) > diff_max:
       # if fit differencess are large, probably error so set max moves equal 0 to 
        # keep cornerpts as is
        max_moves = 0
        rtc = max_moves
        rbc = max_moves
    else: 
        rtc = max_moves
        rbc = max_moves            

    if cornerpts is None and pipeline.lanes.best_cornerpts is not None:
        cornerpts = pipeline.lanes.best_cornerpts

    # set cornerpts adjustment count (max pixel moves allowed per cornerpt)
    cornerpts_adj_ct = (ltc, rtc, rbc, lbc)

    while not stop_optimization and loops < 3:
        stop_optimization, cornerpts, cornerpts_adj_ct, birds_eye_binary  = pipeline.optimize_cornerpts(
            cornerpts, orig_binary, cornerpts_adj_ct)
        loops +=1

    # get fit from optmized binary for finding lane inds and painting road
    l_fit, r_fit = pipeline.get_fits(birds_eye_binary)

    if abs(left_diff) < diff_max and abs(right_diff) < diff_max:
        # only update cornerpts if difference in curve is reasonably small
        pipeline.lanes.update_cornerpts(cornerpts)

        # get nonzero pixels from fit and update lane info
        lx, ly, rx, ry = pipeline.find_lane_inds_from_polyfit(birds_eye_binary, l_fit, r_fit)
        pipeline.lanes.update_lanes(lx, ly, rx, ry)

        # update lane curvature only if cornerpts reasonable
        pipeline.lanes.update_curvature(lx,ly,rx,ry,birds_eye_binary)

        # update line(lane) base position based on cornerpts
        pipeline.lanes.update_line_base_pos(cornerpts)
    
    # color warped road    
    painted_birds_eye_binary = pipeline.paint_road(birds_eye_binary, l_fit, r_fit)
    # pipeline.show_image("painted birds eye binary", painted_birds_eye_binary)

    # unwarp image from bird's view to driver's view    
    unwarped = pipeline.reverse_birds_eye_transform(painted_birds_eye_binary, 
        cornerpts, pipeline.default_dst_pts())
    # pipeline.show_image("unwarped painted road", unwarped)    

    # overlay unwarped color lane to undistorted image original image 
    painted_road = cv2.addWeighted(undistorted_img, 1, unwarped, 0.3, 0)
    # pipeline.show_image("painted road", painted_road)

    # get curvatures of lanes 
    l_curvature, r_curvature = pipeline.lanes.get_curvature()

    if l_curvature is not None and r_curvature is not None:  
        # add curvature info to image if available
        curvature_info = "curvature left: {:.2f} right: {:.2f}".format(
                l_curvature, r_curvature)
        painted_road = pipeline.add_label(painted_road, curvature_info, loc=(50,50)) 
    # get distance from car center to left and right lane
    dist_from_center = pipeline.lanes.get_distance_from_center()
    center_offset_info = "distance from center: {:.2f} m".format(dist_from_center)
    painted_road = pipeline.add_label(painted_road, center_offset_info, loc=(50,100))  
    # pipeline.show_image("colored road", colored_road)

    return painted_road  


output_video_path = 'test_videos_output'

if not os.path.exists(output_video_path):
    os.makedirs(output_video_path)

project_video_output = os.path.join(output_video_path, 'f_painted_project_video.mp4')
project_video_clip = VideoFileClip("../project_video.mp4")#.subclip(0,1)
project_video_clip_colored_lane = project_video_clip.fl_image(paint_lane) 
project_video_clip_colored_lane.write_videofile(project_video_output, audio=False)


error: OpenCV(4.5.3) C:\Users\runneradmin\AppData\Local\Temp\pip-req-build-z4706ql7\opencv\modules\highgui\src\window.cpp:1274: error: (-2:Unspecified error) The function is not implemented. Rebuild the library with Windows, GTK+ 2.x or Cocoa support. If you are on Ubuntu or Debian, install libgtk2.0-dev and pkg-config, then re-run cmake or configure script in function 'cvShowImage'
